# Using GloVe Embedding

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


In this notebook we will leverage Standford's GloVe vectors which is a pretrained embedding on 1.4B Tweets.

Take it easy and pay attention to the main differences with before, and the non-trainable parameters. Finally, check how accurate the results now are.
You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

Follow the instructions. Good luck!


In [1]:
!nvidia-smi

Tue Nov 21 10:04:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.8 MB/s eta 0:00:00
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56439 sha256=4aa6c98295cd535f023835bbc73e5e14213366376d05ab6b3622b1c57cd574

In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False
embedding_dim = 100
epochs=2
batch_size = 500
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2023-11-21 10:05:57--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xds4lua69b7okw8/yelp.csv [following]
--2023-11-21 10:05:57--  https://www.dropbox.com/s/raw/xds4lua69b7okw8/yelp.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0681b34cf15a61e56b7c8283d8.dl.dropboxusercontent.com/cd/0/inline/CH8rkSM5uLN1jqFGlPpVRoNQkosVwF-RQfr0JX1s2kTeXkIyG9eU3LI1PBxXcNZdg7cWFUsPYMEhmxRM1TRBfzHjsGDGdKTRJj3o5LEYyScKkAs5KfZKbA7J2gqYvofNJszpsMaRVfxVYfBddxinWd9u/file# [following]
--2023-11-21 10:05:58--  https://uc0681b34cf15a61e56b7c8283d8.dl.dropboxusercontent.com/cd/0/inline/CH8rkSM5uLN1jqFGlPpVRoNQkosVwF-RQfr0JX1s2kTeXkIyG9eU3LI1PBxXcNZdg7cWFUsPYMEhmxRM1TRBfzHjsGDGdKTRJj3o5LEYyScKk

In [6]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews to have extremes.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [7]:
corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]

In [18]:
path_to_glove_file = "./glove.6B.100d.txt"
embeddings_index = {}
# Construct a function that fills the embedding_index dict for every word in the GloVe file with its coefficients.
# HELP: For that iterate over the Glove file (hint: check that file to view its structure first!), split the word from the numbers, and populate the dictionary with the word and the numbers as a numpy array.
# Hint2: check np.fromstring
def get_coefs(word,*arr):
    return word, np.fromstring(arr[0], sep=' ')

with open(path_to_glove_file, encoding="utf8") as f:
    for line in f:
        coefs = get_coefs(*line.split(maxsplit=1))
        embeddings_index[coefs[0]] = coefs[1]
# FILL

print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
vocab_size = len(tokenizer.word_index) + 1

In [20]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')

First 5 corpus items are ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!', 'I have no idea why some people give bad reviews about this place. It g

In [22]:
hits = 0
misses = 0

embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Create a loop such that for every word in the vocabulary, if it exists in the Glove embedding, then set for that word (that means that index) the tensor of the Glove Embedding. Otherwise fill with 0
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

# FILL

# In the end, the embedding matrix should have, for every word of our tokenizer that exists in GloVe, the tensor representation of that word
print("Converted %d words (%d misses)" % (hits, misses))


Converted 17190 words (2759 misses)


In [23]:
model = Sequential()
# Add the new embedding and set it as non-trainable (although you could fine-tune it if you prefer)
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False))
# Add the same Lambda as before to average out the words dimension
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,)))
# Add a Dense layer to classify words
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1995000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1995101 (7.61 MB)
Trainable params: 101 (404.00 Byte)
Non-trainable params: 1995000 (7.61 MB)
_________________________________________________________________


Notice the Non-trainable parameters! What we are doing is just training the softmax based on correct embeddings. This is called fine tuning the embedding.


In [25]:
def generate_data(corpus, vocab_size, window_size=2, sentence_batch_size=10,  batch_size=250):
    np.random.shuffle(np.array(corpus))
    number_of_sentence_batches = (len(corpus) // sentence_batch_size) + 1
    for batch in range(number_of_sentence_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_sentence_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = []
        Y = []
        for review_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = pad_sequences(contexts, maxlen=maxlen)
                y = to_categorical(labels, vocab_size)
                X.append(x)
                Y.append(y)
        X = tf.constant(X)
        Y = tf.constant(Y)
        number_of_batches = len(X) // batch_size
        for real_batch in range(number_of_batches):
          lower_end = batch*batch_size
          upper_end = (batch+1)*batch_size
          batch_X = tf.squeeze(X[lower_end:upper_end])
          batch_Y = tf.squeeze(Y[lower_end:upper_end])
          yield (batch_X, batch_Y)

In [26]:
# Re implement the method
def fit_model():
    pass

In [27]:
fit_model()

In [28]:
with open('./vectors.txt' ,'w') as f:
    f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
    vectors = model.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))


In [29]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [30]:
w2v.most_similar(positive=['pizza'])

[('sandwich', 0.7488481402397156),
 ('sandwiches', 0.7221847772598267),
 ('bread', 0.700667679309845),
 ('pizzas', 0.7004502415657043),
 ('burger', 0.6977595686912537),
 ('snack', 0.6840099692344666),
 ('taco', 0.6788282990455627),
 ('pie', 0.6776915788650513),
 ('chicken', 0.6714670658111572),
 ('burgers', 0.66903156042099)]

In [31]:
w2v.most_similar(positive=['grape'])

[('grapes', 0.7915149927139282),
 ('wine', 0.7132691144943237),
 ('varieties', 0.7115378975868225),
 ('chardonnay', 0.7091464996337891),
 ('wines', 0.7076444029808044),
 ('pinot', 0.7055837512016296),
 ('zinfandel', 0.7001710534095764),
 ('vines', 0.681800127029419),
 ('sauvignon', 0.6581584811210632),
 ('cabernet', 0.6564268469810486)]

Do you notice the difference in the accuracy? For any task first search if there are any pretrained models to use!